In [2]:
import pandas as pd
import networkx as nx
import numpy as np
import torch

## 导入数据

In [3]:
data = pd.read_csv('D:\\学习\\论文\\2024new-第二篇\\pmy\\dataset\\NF-BoT-IoT\\data\\NF-BoT-IoT.csv')

## 将每个（源IP 目的IP）的netflow数量对齐至最大长度 max_length

In [7]:
max_length = 10
filtered_IP_list = []
# 提取 sourceIP 和 dstIP 的所有唯一组合
ip_combinations = data[['IPV4_SRC_ADDR', 'IPV4_DST_ADDR']].drop_duplicates()
# 遍历每个组合
for _, row in ip_combinations.iterrows():
    source_ip = row['IPV4_SRC_ADDR']
    dst_ip = row['IPV4_DST_ADDR']
    # 筛选出匹配的行
    filtered_data = data[(data['IPV4_SRC_ADDR'] == source_ip) & (data['IPV4_DST_ADDR'] == dst_ip)]
    # 将筛选结果存入数组（包含元信息）
    filtered_IP_list.append({
        "IPV4_SRC_ADDR": source_ip,
        "IPV4_DST_ADDR": dst_ip,
        "data": filtered_data
    })

# 找出每个 IP 组的 data 个数
data_lengths = [len(group["data"]) for group in filtered_IP_list]


if (max_length == 0): # 没有提前说明最大长度
    max_length = max(data_lengths)



# 在随机位置插入空值
for group in filtered_IP_list:
    data = group["data"]
    current_length = len(data)

    # 如果当前数据长度小于 max_length，补齐为 max_length 行（全零行替换）
    if current_length < max_length:
        # 随机生成替换索引
        # replace_indices = np.random.choice(range(max_length), size=current_length, replace=False)
        replace_indices = np.sort(np.random.choice(range(max_length), size=current_length, replace=False))

        # 构建全零行 DataFrame
        zero_rows = pd.DataFrame(0, index=range(max_length), columns=data.columns)

        # 创建新的 DataFrame，将全零行替换到指定位置
        full_data = zero_rows.copy()
        # for idx, row in enumerate(data.itertuples(index=False)):
        #     full_data.iloc[replace_indices[idx]] = row
        for idx, row in enumerate(data.itertuples(index=False)):
            full_data.iloc[replace_indices[idx]] = pd.DataFrame([row])  # 仅保留数值部分
        # 更新 group 数据
        group["data"] = full_data.head(max_length)
    else:
        # 更新 group 数据
        group["data"] = data.head(max_length)
    

## 按照当前填充过的数据集，重新构建数据集格式
每个小数据集，包含所有不同的（源IP、目的IP）的一条数据

In [10]:
FinalDataList = []
for i in range(max_length):
    combined_data = []
    # 遍历 filtered_data_list
    for group in filtered_IP_list:
        source_ip = group["IPV4_SRC_ADDR"]
        dst_ip = group["IPV4_DST_ADDR"]
        data = group["data"]

        # 从每组数据中提取一行（例如：第一行）
        extracted_row = data.iloc[i]  # 或 np.random.choice(len(data)) 选择随机行
        combined_data.append(extracted_row)

        # 添加到 最终的数据集
        FinalDataList.append({
            "source_ip": source_ip,
            "dst_ip": dst_ip,
            "data": extracted_row.to_frame().T  # 转换为 DataFrame 格式
        })

In [11]:
FinalDataList

[{'source_ip': '192.168.100.6',
  'dst_ip': '192.168.100.1',
  'data':    IPV4_SRC_ADDR L4_SRC_PORT  IPV4_DST_ADDR L4_DST_PORT PROTOCOL L7_PROTO  \
  0  192.168.100.6       52670  192.168.100.1          53       17    5.212   
  
    IN_BYTES OUT_BYTES IN_PKTS OUT_PKTS TCP_FLAGS FLOW_DURATION_MILLISECONDS  \
  0       71       126       1        1         0                    4294966   
  
    Label  Attack  
  0     0  Benign  },
 {'source_ip': '192.168.100.6',
  'dst_ip': '192.168.100.1',
  'data':       IPV4_SRC_ADDR L4_SRC_PORT  IPV4_DST_ADDR L4_DST_PORT PROTOCOL L7_PROTO  \
  8394  192.168.100.6       58727  192.168.100.1          53       17    5.212   
  
       IN_BYTES OUT_BYTES IN_PKTS OUT_PKTS TCP_FLAGS FLOW_DURATION_MILLISECONDS  \
  8394       71       126       1        1         0                    4294957   
  
       Label  Attack  
  8394     0  Benign  },
 {'source_ip': '192.168.100.6',
  'dst_ip': '192.168.100.1',
  'data':        IPV4_SRC_ADDR L4_SRC_PORT  IPV4_DS

In [16]:
from GraphBuild import *
snapshots = build_graph(FinalDataList)

AttributeError: 'list' object has no attribute 'iterrows'